In [ ]:
# This cell is only for setting example parameter defaults - gets replaced by sidecar.
day_obs = "20260128"
#day_obs = "20251108"
#day_obs = "today"
#telescope = "AuxTel"  
telescope = "SimonyiTel"
timezone = "Chile/Continental"
#timezone = "UTC"

# Scheduler: Targets, Observations and ConsDB Visits for {{ params.day_obs }} {{ params.telescope }}

The Feature Based Scheduler requests `Targets` and completed observation scripts result in `Observations`; these are both are tracked in the EFD.
The Consdb reports acquired `Visits`.  
On-sky exposures can also be acquired directly through execution of scripts or JSON BLOCKs; these don't result in `Targets` but do produce `Visits`.


* [Almanac](#Almanac)
* [EFD Configuration](#EFD_configuration)
* [Targets and Visits](#Targets_visits)
* [Overheads](#Overheads)
* [Summary Plot](#Summary_plot)

In [ ]:
# import os
# if os.getenv("EXTERNAL_INSTANCE_URL") is not None:
#     print("updating rubin_nights")
#     !pip install --user --upgrade git+https://github.com/lsst-sims/rubin_nights.git  --no-deps  > /dev/null 2>&1

In [ ]:
import os
import warnings
import copy
from math import floor
import numpy as np
import healpy as hp
import matplotlib.pylab as plt
from cycler import cycler
import colorcet as cc

import pandas as pd
from pandas import option_context
from IPython.display import display, Markdown, HTML

import datetime
from astropy.time import Time, TimeDelta
import astropy.units as u
import astropy
astropy.utils.iers.conf.iers_degraded_accuracy = 'ignore'

import datetime
import pytz

import datetime
from zoneinfo import ZoneInfo

try:
    tz = ZoneInfo(timezone)
    tz_utc = ZoneInfo("UTC")
except ZoneInfoNotFoundError:
    print("Timezone should be a string recognizable to `ZoneInfo`.")
    print("Using Chile/Continental (+UTC) backup.")
    tz = ZoneInfo("Chile/Continental")
    tz_utc = ZoneInfo("UTC")

from rubin_scheduler.site_models import Almanac
from rubin_scheduler.utils import Site
from rubin_scheduler.scheduler.model_observatory import KinemModel, tma_movement, rotator_movement

from rubin_nights import connections, scriptqueue, observatory_status
import rubin_nights.rubin_scheduler_addons as rn_sch
import rubin_nights.dayobs_utils as rn_dayobs
from rubin_nights.targets_and_visits import targets_and_visits
from rubin_nights.augment_visits import augment_visits

# Where is the notebook running? (RSPs are 'special')
current_location = os.getenv("EXTERNAL_INSTANCE_URL", "")

# RUBIN_SIM_DATA_DIR at usdf
if 'usdf' in current_location:
    os.environ["RUBIN_SIM_DATA_DIR"] = "/sdf/data/rubin/shared/rubin_sim_data"

# TOKEN CONFIGURATION
if current_location != "":
    # You are on an rsp.
    # You should use the default RSP values, whether summit/base/USDF.
    tokenfile = None
    site = None
# If you are outside of an RSP? - just use USDF and your own USDF-RSP token
# See https://rsp.lsst.io/guides/auth/creating-user-tokens.html
else:
    # Substitute the location of your own tokenfile
    tokenfile = os.getenv("ACCESS_TOKEN_FILE", "")
    site = os.getenv("DATA_SITE", "")
    if tokenfile == "":
        # A very reasonable backup.
        tokenfile = os.path.join(os.path.expanduser("~"), ".lsst/usdf_rsp")
        site = 'usdf'

minutes_to_days = 1./60/24
seconds_to_days = 1./60/60/24

%matplotlib inline

In [ ]:
endpoints = connections.get_clients(tokenfile=tokenfile, site=site)

In [ ]:
if isinstance(day_obs, str):
    if day_obs.lower() == "today":
        day_obs = rn_dayobs.today_day_obs()
    elif day_obs.lower() == "yesterday":
        day_obs = rn_dayobs.yesterday_day_obs()

try:
    int(day_obs)
    day_obs = rn_dayobs.day_obs_int_to_str(day_obs)
except ValueError:
    pass
        
day_obs_time = Time(f"{day_obs}T12:00:00", format='isot', scale='tai')

t_start = Time(f"{day_obs}T12:00:00", format='isot', scale='tai')
t_end = Time(f"{day_obs}T12:00:00", format='isot', scale='tai') + TimeDelta(1, format='jd')

In [ ]:
# Use Telescope to set Queue/salIndex
# Use Telescope + dayObs to set Instrument (based on date range when lsstcomcam was on-sky)

comcam_end = Time("2024-12-12T12:00:00", format='isot', scale='utc')

match telescope.lower():
    case "simonyi" | "simonyitel":
        salindex = 1
        instrument = "unknown"
    case "auxtel":
        salindex = 2
        instrument = "latiss"
        program = ["BLOCK-311", "BLOCK-312"]
    case "ocs": 
        salindex = 3
        instrument = "unknown"
    case _:
        raise ValueError("Telescope should be among Simonyi, Auxtel or OCS")

if instrument == "unknown":
    if Time(day_obs, format='isot', scale='utc') < comcam_end:
        instrument = "lsstcomcam"
        program = ["BLOCK-320"]
    else:
        instrument = "lsstcam"
        program = ["BLOCK-365", "BLOCK-407", "BLOCK-408", "BLOCK-416", "BLOCK-417", "BLOCK-419", "BLOCK-421"]

print(f"Checking for targets issued on salIndex {salindex} taken with instrument {instrument}")
print(f"Science programs include {program} visits")

<a id="Almanac"></a>

In [ ]:
## Almanac ## 

display(Markdown(f"## Almanac information for dayobs {day_obs}"))
site = Site('LSST')
almanac = Almanac()
night_events = almanac.get_sunset_info(evening_date=day_obs, longitude=site.longitude_rad)
sunset_12 = Time(night_events['sun_n12_setting'], format='mjd', scale='utc') 
sunrise_12 = Time(night_events['sun_n12_rising'], format='mjd', scale='utc')
sunset = Time(night_events['sunset'], format='mjd', scale='utc') 
sunrise = Time(night_events['sunrise'], format='mjd', scale='utc')
night_length = sunrise_12.mjd - sunset_12.mjd

display(Markdown(f"0-deg sunset at {sunset.to_datetime(timezone=tz_utc).strftime('%x %X')} UTC  -- {sunset.to_datetime(timezone=tz).strftime('%x %X')} {timezone}"))
display(Markdown(f"0-deg sunrise at {sunrise.to_datetime(timezone=tz_utc).strftime('%x %X')} UTC --  {sunrise.to_datetime(timezone=tz).strftime('%x %X')} {timezone}"))
display(Markdown(f"12-deg sunset at {sunset_12.to_datetime(timezone=tz_utc).strftime('%x %X')} UTC  -- {sunset.to_datetime(timezone=tz).strftime('%x %X')} {timezone}"))
display(Markdown(f"12-deg sunrise at {sunrise_12.to_datetime(timezone=tz_utc).strftime('%x %X')} UTC --  {sunrise.to_datetime(timezone=tz).strftime('%x %X')} {timezone}"))
display(Markdown(f"allowing for a (-12deg) night of {night_length * 24 :.2f} hours"))
moon_phase = almanac.get_sun_moon_positions(sunset_12.mjd)['moon_phase']
if not np.isnan(night_events['moonrise']):
    moonrise = Time(night_events['moonrise'], format='mjd', scale='utc')
    display(Markdown(f"Moonrise is at {moonrise.to_datetime(timezone=tz_utc).strftime('%x %X')} UTC -- {moonrise.to_datetime(timezone=tz).strftime('%x %X')} {timezone}"))
if not np.isnan(night_events['moonset']):
    moonset = Time(night_events['moonset'], format='mjd', scale='utc')
    display(Markdown(f"Moonset at {moonset.to_datetime(timezone=tz_utc).strftime('%x %X')} UTC -- {moonset.to_datetime(timezone=tz).strftime('%x %X')} {timezone}"))
display(Markdown(f"Moon phase is {moon_phase :.1f} (0=new, 100=full)."))

In [ ]:
# Report time notebook was run, which is likely useful if running notebook in the middle or start of the night
current_time = Time.now()
display(Markdown(f"Time of notebook execution: {current_time.isot}"))
if current_time > sunrise:
    display(Markdown("Night is complete."))
elif current_time < sunrise and current_time > sunset:
    display(Markdown("Night is in progress."))
elif current_time < sunset:
    display(Markdown("Night not yet started."))

<a id="EFD_configuration"></a>

## Scheduler Configuration Information

In [ ]:
# What versions of the Scheduler modules are being used
display(Markdown("### Scheduler Versions"))

dd = scriptqueue.get_scheduler_configs(sunset, sunrise, endpoints['efd'], endpoints['obsenv'], queue_index=salindex)
dd = dd.query("classname != 'Obsenv Check'")
cols = ['salIndex', 'classname', 'description', 'config']
display(HTML(dd[cols].to_html(escape=False)))


# What other BLOCKS have been requested in this night (outside the FBS)
display(Markdown("### JSON BLOCKS"))
#topic = 'lsst.sal.Scheduler.command_addBlock'
topic = 'lsst.sal.Scheduler.logevent_blockStatus'
fields = endpoints['efd'].get_fields(topic)
fields = [f for f in fields if 'private' not in f]
dd = endpoints['efd'].select_time_series(topic, fields, sunset, sunrise, index=salindex)
if len(dd) == 0:
    print(f"No JSON BLOCKS added between {sunset.iso} and {sunrise.iso}")
else:
    grouped_dd = dd.groupby('id')[['id', 'definition', 'executionsCompleted', 'hash', 'salIndex']].agg('max')
    grouped_dd_start = dd.groupby('id')[['id', 'definition', 'executionsCompleted', 'hash', 'salIndex']].agg('min')
    grouped_dd['night_executions']  = grouped_dd['executionsCompleted'] - grouped_dd_start['executionsCompleted']
    display(grouped_dd[['id', 'definition', 'hash', 'executionsCompleted', 'night_executions']])

In [ ]:
# Your JIRA Cloud base URL for API
ZEPHYR_BASE_URL = "https://api.zephyrscale.smartbear.com/v2/" + "testcases/"
JIRA_BASE_URL = "https://rubinobs.atlassian.net/rest/api/2/" + "issue/"

testcase_base_url = "https://rubinobs.atlassian.net/projects/BLOCK?selectedItem=com.atlassian.plugins.atlassian-connect-plugin:com.kanoah.test-manager__main-project-page#!/v2/testCase/"

jiraticket_base_url = "https://rubinobs.atlassian.net/browse/"

if len(dd) > 0:
    display(Markdown("For more information of test cases and blocks:"))
    for block in grouped_dd.id:
        if block is not None and block.startswith("BLOCK-T"):
            url = testcase_base_url + block
        elif block is not None and block.startswith("BLOCK-"):
            url = jiraticket_base_url + block
        display(Markdown(f'<a href="{url}" target="_blank" rel="noreferrer noopener">{block}</a>'))

In [ ]:
if salindex == 2:
    pass

else:

    display(Markdown("### Observatory slew speed"))
    
    # use 1% tma movement to scale other values
    one_percent = tma_movement(1)
    
    tma = observatory_status.get_tma_limits(sunset, sunrise, endpoints['efd'])
    display(tma)
    
    alt_percent = tma.altitude_maxspeed.values / one_percent['altitude_maxspeed']
    az_percent = tma.azimuth_maxspeed.values /one_percent['azimuth_maxspeed']
    
    print(f"Sunset/sunrise {sunset.iso} {sunrise.iso}")
    print(f"Altitude movement {alt_percent} percent of max")
    print(f"Azimuth movement {az_percent} percent of max")
    #rotator = observatory_status.get_rotator_limits(sunset, sunrise, endpoints['efd'])
    #display(rotator) -- reports a lot, but is always just 100%

<a id="Targets_visits"></a>

## Fetch and link Targets, Observations, and Visits 

In [ ]:
# Fetch joined targets and visits, along with joined targets + observations, and nextvisits + visits, and all visits
target_visits, cols, target_obs, nextvisit_visit, visits = targets_and_visits(sunset, sunrise, endpoints, queue_index=salindex)

if target_visits is None:
    target_visits = []
    target_obs = []

if len(target_visits) == 0:
    print("Could not find target+visit data for this night.")

print(f"Found {len(visits)} visits.")

In [ ]:
if len(target_obs) > 0:
    n_snapshots = len(target_obs.snapshotUri.unique())
else:
    n_snapshots = 0
print(f"Found {len(target_obs)} targets and observations, with {n_snapshots} snapshots.")

if len(visits) > 0:
    visits_sci = visits.query("science_program in @program")
    print(f"Found {len(visits)} visits, {len(visits_sci)} in science program {program}")

    
    if instrument == 'lsstcam':
        visits_sci = visits.query("science_program in @program and "\
                                  "observation_reason != 'field_survey_science' and "\
                                  "observation_reason != 'block-t538' and "\
                                  "not observation_reason.str.contains('close_loop')")
        print(f"Found {len(visits_sci)} science visits (not counting close_loop)")
    else:
        visits_sci = pd.DataFrame([], columns=visits.columns)

    if len(target_visits) > 0:
        print(f"Matched {len(target_visits.query("visit_id > 0"))} targets to visits, {len(target_obs[~target_obs['time_o'].isna()])} targets to observations")
    else:
        print("Found 0 target-observations-visits (must be generated from the FBS).")

    if len(visits_sci) == 0:
        print(f"Only found visits from blocks {visits.science_program.unique()}")

# Fetching snapshots directly gets better time maybe? 
topic = "lsst.sal.Scheduler.logevent_largeFileObjectAvailable"
fields = ["url"]
snapshots = endpoints['efd'].select_time_series(topic, fields, sunset, sunrise, index=salindex)
if len(snapshots) != n_snapshots:
    print(f"hmm - Found different number of snapshots from {topic} ({len(snapshots)} than from targets {n_snapshots})")

In [ ]:
if len(visits) > 0 and salindex != 2:
    # Use the full visits to check slew times 
    add_to_model_slew = 3.5
    visitsS, slewing = rn_sch.add_model_slew_times(visits, endpoints['efd'], model_settle=add_to_model_slew, dome_crawl=True)
    mt_slews = observatory_status.mtm1m3_slewflag_times(sunset, sunrise, endpoints['efd'])
    add_to_mt_slew = 1.5
    mt_slews['mt_slew_time'] += add_to_mt_slew
    visitsS = visitsS.merge(mt_slews, how='left', left_on='group_id', right_on='groupId')
    if len(target_visits) > 0:
        # Match back to targets + visits
        target_visits_slew = pd.merge(target_visits, visitsS[['visit_id', 'slew_model', 'slew_model_ideal', 'mt_slew_time', 'slew_distance', 'model_gap']], left_on='visit_id', right_on='visit_id')    
elif salindex == 2:
    visitsS = visits
    target_visits_slew = target_visits
else:
    visitsS = []
    target_visits_slew = []

In [ ]:
c = None
if len(target_visits) > 0:
    groupcols = ['science_program', 'img_type', 'target_name', 'observation_reason', 'day_obs', 'visit_id'] 
    c = visits[groupcols].groupby(['science_program', 'img_type']).agg({'science_program' : ['first'],
                                                                        'target_name' : ['unique'], 
                                                                        'observation_reason' : ['unique'],
                                                                        'day_obs' : ['nunique'],
                                                                        'visit_id' : ['first', 'last', 'count']})
    display(Markdown(f"ConsDB Visits"))
    with option_context('display.max_colwidth', None):
        display(HTML(c.sort_values(by=('visit_id', 'first')).to_html()))

In [ ]:
#target_visits_slew.query("observation_reason.str.contains('close_loop')")[['seq_num', 'time_target', 'time_nextvisit', 'time_observation', 'obs_start', 'group_id', 'scriptSalIndex', 'observation_reason', 'scheduler_note']]

<a id="Overheads"></a>

## Overheads and Issues

Look at actual visit gaps and slewtime from mtm1m3 slew flags, as well as slew model.

In [ ]:
slew_error_under = -1.0
slew_error_over = 10.0

# Overview of slew flag vs. model differences
if len(target_visits) > 0 and salindex != 2:
    fig, ax = plt.subplots(1, 2, figsize=(15, 5))
    bins = np.arange(-2, 25, 0.4)
    _ = ax[0].hist(target_visits_slew['visit_gap'], bins=bins, alpha=0.6, histtype='bar', label="Visit gap")
    _ = ax[0].hist(target_visits_slew['mt_slew_time'], bins=bins, alpha=0.6, histtype='bar', label="MT slew flag")
    _ = ax[0].hist(target_visits_slew['slew_model'], bins=bins, alpha=0.6, histtype='bar', label="Model slew")
    ax[0].legend(loc=(0.7, 0.8))
    ax[0].set_xlabel("Seconds", fontsize='large')
    ax[0].grid(alpha=0.3)

    bins = np.arange(-2, 8, 0.1)
    slew_gap = target_visits_slew['visit_gap'] - target_visits_slew['mt_slew_time']
    model_gap = target_visits_slew['visit_gap'] - target_visits_slew['slew_model']
    _ = ax[1].hist(slew_gap, bins, alpha=0.6, histtype='bar', label=f"Visit gap - MT slew ({slew_gap.median():.1f}s/{slew_gap.mean():.1f}s))")
    _ = ax[1].hist(model_gap, bins, alpha=0.6, histtype='bar', label=f"Visit gap - model ({model_gap.median():.1f}s/{model_gap.mean():.1f}s)")
    ax[1].axvline(slew_error_under, color='pink', linestyle=':')
    ax[1].axvline(slew_error_over, color='pink', linestyle=':')
    ax[1].legend(loc=(0.44, 0.8))
    ax[1].set_xlabel("Seconds", fontsize='large')
    ax[1].grid(alpha=0.3)

    print(f"{day_obs}")
    print(f"Model slew includes additional overhead of {add_to_model_slew}s ; {add_to_mt_slew}s is added to mtslew flag time.")
    print(f"{len(target_visits_slew)} slews")
    print(f"{len(np.where(np.isnan(target_visits_slew['mt_slew_time']))[0])} mt slew flags give NaN slew times")
    print(f"{len(np.where(np.isnan(target_visits_slew['slew_model']))[0])} model slews are Nan")

In [ ]:
if len(target_visits_slew) > 0:
    min_y = -2
    max_y = 10
    fig, axes = plt.subplots(2,1, figsize=(12, 5), sharex=True)
    ax = axes[0]
    q = target_visits_slew
    ax.plot(q.seq_num, q.visit_gap - q.mt_slew_time, linestyle=':', marker='.', color='k', label="Actual visit gap - MTSlewFlag")
    #ax.plot(Time(q.obs_start_mjd, format='mjd').datetime, q.visit_gap - q.mt_slew_time, linestyle=':', marker='.', color='k', label="Actual visit gap - MTSlewFlag")
    ax.legend(loc=(1.01, 0.5))
    ax.set_ylim(min_y, max_y)
    ax.set_ylabel("seconds")
    ax.grid(alpha=0.5)
    ax = axes[1]
    ax.plot(q.seq_num, q.visit_gap - q.slew_model, linestyle=':', marker='.', color='k', label="Actual visit gap - Predicted gap")
    #ax.plot(Time(q.obs_start_mjd, format='mjd').datetime, q.visit_gap - q.slew_model, linestyle=':', marker='.', color='k', label="Actual visit gap - Predicted gap")
    ax.legend(loc=(1.01, 0.5))
    ax.set_ylim(min_y, max_y)
    ax.set_xlabel("seq_num")
    ax.set_ylabel("seconds")
    ax.grid(alpha=0.5)
    fig.suptitle(f"Day_obs {day_obs}")
    fig.tight_layout()

In [ ]:
print("This compares the predicted (model) visit gap to the actual visit gap, to see any issues with the slew model.")
# How do we want to identify gaps? 
# slew models are not symmetrically 'wrong'
if len(target_visits_slew) > 0 and salindex != 2:
    q = target_visits_slew
    fig, ax = plt.subplots(1, 2, figsize=(15, 5))
    x = np.arange(q.slew_model.min(), q.slew_model.max())
    if x.max() > 100:
        ax[0].axhline(120, color='pink', linestyle=':')
    ax[0].plot(x, x, color='gray', linestyle=':')
    ax[0].plot(x+slew_error_under, x, color='gray', linestyle=':')
    ax[0].plot(x+slew_error_over, x, color='gray', linestyle=':')
    ax[0].fill_betweenx(x, x+slew_error_under, x+slew_error_over, color='pink', alpha=0.2, zorder=0)
    ax[0].plot(q.visit_gap, q.slew_model, 'k.')
    ax[0].set_xlabel("Visit Gap (seconds)", fontsize='large')
    ax[0].set_ylabel("Predicted visit gap (seconds)", fontsize='large')
    
    ax[1].plot(x, x, color='gray', linestyle=':')
    ax[1].plot(x+slew_error_under, x, color='gray', linestyle=':')
    ax[1].plot(x+slew_error_over, x, color='gray', linestyle=':')
    ax[1].fill_betweenx(x, x+slew_error_under, x+slew_error_over, color='pink', alpha=0.2, zorder=0)
    ax[1].plot(q.visit_gap, q.slew_model, 'k.')
    ax[1].set_xlabel("Visit Gap (seconds)", fontsize='large')
    ax[1].set_ylabel("Predicted visit gap (seconds)", fontsize='large')
    ax[1].set_xlim(2, 30)
    ax[1].set_ylim(2, 30)

In [ ]:
# cols = ['visit_id', 'obs_start', 's_ra', 's_dec', 'sky_rotation', 'band', 
#             'altitude', 'azimuth', 'airmass', 'clouds', 'observation_reason', 'target_name', 
#             'slew_model', 'slew_model_ideal', 'mt_slew_time', 'visit_gap', 'slew_distance', 'model_gap']
# target_visits_slew.query("model_gap < -1.5")[cols]
# filter_changes = np.where(visits.band[1:].values != visits.band[:-1].values)[0]
# filter_changes = filter_changes + 1
# fchanges = visits.iloc[filter_changes]
# len(fchanges), (fchanges.obs_start_mjd.max() - fchanges.obs_start_mjd.min()) * 24
# deltas = np.diff(fchanges.obs_start_mjd)*24*60
# plt.plot(Time(fchanges.obs_start_mjd.iloc[1:], format='mjd', scale='tai').to_datetime(), deltas, 'k.')
# _ = plt.xticks(rotation=90)
# deltas
# delta_seq = np.diff(fchanges.seq_num)
# [f"{ds} {dt:.2f}" for ds, dt in zip(delta_seq, deltas)]

In [ ]:
if len(target_visits)>  0 and salindex != 2:
    good = np.where((target_visits_slew.model_gap <= slew_error_over) & (target_visits_slew.model_gap >= slew_error_under), True, False)
    target_visits_slew['good'] = good
    # science survey without outliers 
    q = target_visits_slew.loc[np.where(good)].query("visit_id in @visits_sci.visit_id")
    n_good = len(q)
    q = target_visits_slew.loc[np.where(~good)].query("visit_id in @visits_sci.visit_id")
    n_outliers = len(q)
    print(f"Number of total visits (from FBS) {len(target_visits_slew)}; number with outlier times: {len(target_visits_slew.query("good == False"))}")
    print(f"Number of sci {program} survey visits {len(visits_sci)}; number with outlier times: {n_outliers}")
    print(f"Outliers identified by visit gaps more than {slew_error_under} seconds or {slew_error_over} seconds away from the slew model estimate.")
    print("")
    print(f"All FBS visits: Median visit gap {target_visits_slew.visit_gap.median() :.1f}; average visit gap {target_visits_slew.visit_gap.mean():.1f}")
    # remove outliers and estimate
    q = target_visits_slew.loc[np.where(good)]
    print(f"Without outliers: Median visit gap {q.visit_gap.median() :.1f}; average visit gap {q.visit_gap.mean() :.1f}")
    print("")
    # SV survey without outliers
    q = target_visits_slew.query("visit_id in @visits_sci.visit_id")
    n_good = len(q)
    print(f"All sci survey: Median visit gap {q.visit_gap.median() :.1f}; average visit gap {q.visit_gap.mean() :.1f}")
    q = target_visits_slew.loc[np.where(good)].query("visit_id in @visits_sci.visit_id")
    n_good = len(q)
    print(f"Sci survey without outliers: Median visit gap {q.visit_gap.median() :.1f}; average visit gap {q.visit_gap.mean() :.1f}")
    q = target_visits_slew.loc[np.where(~good)].query("visit_id in @visits_sci.visit_id")
    n_outliers = len(q)
    print(f"Sci survey outliers only: Median visit gap {q.visit_gap.median() :.1f}; average visit gap {q.visit_gap.mean() :.1f}")


In [ ]:
if len(target_visits_slew) > 0:
    # compare to slew times from 'ideal' model (same as v5.1), without outliers
    print("Considering only the visits with visit_gap within the limited expected range")
    tt = target_visits_slew.query("good == True")
    
    # Calculate "open shutter fraction" without faults
    slew_eff = (tt.exp_time.sum()) / (tt.dark_time.sum() + tt.visit_gap.sum())
    ideal_eff = (tt.exp_time.sum()) / (tt.dark_time.sum() + tt.slew_model_ideal.sum())
    print(f"Slew efficiency factor: {slew_eff: 0.2f}")
    print(f"Ideal model efficiency equivalent: {ideal_eff: 0.2f}")
    print(f"Ratio - slew / ideal {slew_eff / ideal_eff :0.2f}")

In [ ]:
breaks = []
if len(visitsS) > 0 and salindex != 2:
    bidx = visitsS.query("model_gap > @slew_error_over").index
    for bi in bidx: 
        if bi == 0:
            continue
        b_start = visitsS.iloc[bi-1].obs_end_mjd
        b_end = visitsS.iloc[bi].obs_start_mjd
        if visitsS.iloc[bi].science_program in program:
            breaks.append([b_start, b_end])

<a id="Summary_plot"></a>

## Summary Plot of FBS Targets, Observations, and ConsDB Visits

In [ ]:
def mjd_to_datetime(mjd, scale='utc', timezone=tz):
    return Time(mjd, format='mjd', scale=scale).utc.to_datetime(timezone=timezone)

In [ ]:
eps = 1
fig, ax = plt.subplots(figsize=(13, 8))
ax_utc = ax.twiny()

ax.set_title(f"{telescope} DAYOBS {day_obs}", pad=20)

# Shade astronomical events
ax.fill_between([mjd_to_datetime(night_events['sun_n12_setting']), 
                  mjd_to_datetime(night_events['sun_n18_setting'])],
                 2.5, 0.0, color='lightgray', alpha=0.3)
ax.fill_between([mjd_to_datetime(night_events['sunset']), 
                 mjd_to_datetime(night_events['sun_n12_setting'])], 
                  2.5, 0.0, color='gray', alpha=0.3)
ax.fill_between([mjd_to_datetime(night_events['sun_n18_rising']), 
                 mjd_to_datetime(night_events['sun_n12_rising'])],
                 2.5, 0.0, color='lightgray', alpha=0.3)
ax.fill_between([mjd_to_datetime(night_events['sun_n12_rising']), 
                 mjd_to_datetime(night_events['sunrise'])],
                 2.5, 0.0, color='gray', alpha=0.3)

if not np.isnan(night_events['moonrise']):
    ax.axvline(mjd_to_datetime(night_events['moonrise']), linestyle='-', color='blue', alpha=0.3)
if not np.isnan(night_events['moonset']):
    ax.axvline(mjd_to_datetime(night_events['moonset']), linestyle='-', color='red', alpha=0.3)

if salindex == 2:
    plot_key = 'target_name'
else:
    plot_key = 'science_program'

colors = cc.glasbey_category10
# Assign distinct target sets with different colors
marker_colors = {}
labels = {}
count = 0
if len(visitsS) > 0:
    for sp in visitsS[plot_key].unique():
        if sp in program:
            q = visitsS.query("science_program == @sp")
            for obs_reason in q.observation_reason.unique():
                marker_colors[obs_reason] = colors[count]
                labels[obs_reason] = obs_reason
                count += 1
        else:
            marker_colors[sp] = colors[count]
            labels[sp] = sp
            count += 1


# Plot snapshot times
if len(snapshots) > 0:
    snapshot_times = Time(snapshots.index.values, scale='utc').to_datetime()
    ax.plot(snapshot_times, np.ones(len(snapshots)) * 1.01, color='orange', 
            marker='|', markersize=13, linestyle='')

if len(breaks) > 0:
     # Shade breaks in observation events
    for break_count, b in enumerate(breaks):
        ax.fill_between(mjd_to_datetime(b), 2.5, 0.0, color='pink', alpha=0.3)
        ax.text(mjd_to_datetime(b[0]), 0.97, f"D-{break_count}", fontsize='small', rotation=90)

if len(visitsS) > 0:
    visit_alpha = 0.7
    marker_dict = {'science': 'o', 
                   'acq': '*', 
                   'cwfs' : 'H',
                   'focus': 'D'}
    for imgtype in marker_dict:
        q = visitsS.query("img_type == @imgtype")
        for sp in q[plot_key].unique():
            qq = q.query(f"{plot_key} == @sp")
            if sp in program: # only simonyi
                for obs_reason in qq.observation_reason.unique():
                    qq = q.query("science_program == @sp and observation_reason == @obs_reason")
                    if imgtype == 'science':
                        label = obs_reason
                    else:
                        label = None
                    ax.plot(mjd_to_datetime(qq.obs_start_mjd, 'tai'), qq.airmass, 
                            marker=marker_dict[imgtype], linestyle='',
                            color=marker_colors[obs_reason], label=label,
                            alpha=visit_alpha, markerfacecolor='none', zorder=3)
            else:
                if imgtype == 'science':
                    label = sp
                elif imgtype == 'acq' and len(visitsS.query(f"{plot_key} == @sp and img_type == 'science'")) == 0:
                    label = sp
                else:
                    label = None
                ax.plot(mjd_to_datetime(qq.obs_start_mjd, 'tai'), qq.airmass, 
                        marker=marker_dict[imgtype], linestyle='',
                        color=marker_colors[sp], label=label,
                        alpha=visit_alpha, markerfacecolor='none', zorder=3)

if len(target_visits) > 0:
    marker_dict = {'complete': '.', 'incomplete': '+'} 
    for marker in marker_dict:
        if marker == 'incomplete':
            q = target_visits_slew.loc[target_visits_slew['time_observation'].isna()]
            tlabel = 'Target only '
        else:
            q = target_visits_slew.loc[~target_visits_slew['time_observation'].isna()]
            tlabel = 'Target+obs '
        for sp in q.science_program.unique():
            qq = q.query(f"{plot_key} == @sp")
            if len(qq) > 0:
                if sp in program:
                    for obs_reason in qq.observation_reason.unique():
                        label = tlabel + obs_reason
                        qq = q.query("science_program == @sp and observation_reason == @obs_reason")
                        times = Time(qq.time_target, scale='utc')
                        ax.plot(mjd_to_datetime(times.mjd), qq.airmass, 
                                marker=marker_dict[marker], linestyle='',
                                color=marker_colors[obs_reason], alpha=0.8, label=label)
                else:
                    label = tlabel + str(sp)
                    times = Time(qq.time_target, scale='utc')
                    ax.plot(mjd_to_datetime(times.mjd, 'utc'), qq.airmass, 
                            marker=marker_dict[marker], linestyle='',
                            color=marker_colors[sp], label=label)


ax.legend(loc=(1.01, 0.0), ncol=2)

x0 = night_events['sunset']+30/60/24

ax.set_xlim(mjd_to_datetime(night_events['sunset']+30/60/24), 
         mjd_to_datetime(night_events['sunrise']-30/60/24))
ax_utc.set_xlim(mjd_to_datetime(night_events['sunset']+30/60/24, 'utc', timezone=tz_utc), 
         mjd_to_datetime(night_events['sunrise']-30/60/24, 'utc', timezone=tz_utc))

ax.set_xlim(mjd_to_datetime(night_events['sunset']+30/60/24), 
         mjd_to_datetime(night_events['sunrise']-30/60/24))
ax_utc.set_xlim(mjd_to_datetime(night_events['sunset']+30/60/24, 'utc', timezone=tz_utc), 
         mjd_to_datetime(night_events['sunrise']-30/60/24, 'utc', timezone=tz_utc))

# Set ticks relevant sides
ax.tick_params(axis="x", bottom=True, top=False, labelbottom=True, labeltop=False)
ax_utc.tick_params(axis="x", bottom=False, top=True, labelbottom=False, labeltop=True)

# Rotate and align bottom ticklabels
plt.setp([tick.label1 for tick in ax.xaxis.get_major_ticks()], rotation=45,
         ha="right", va="center", rotation_mode="anchor")

# Rotate and align top ticklabels
plt.setp([tick.label2 for tick in ax_utc.xaxis.get_major_ticks()], rotation=45,
         ha="left", va="center",rotation_mode="anchor")

plt.grid(True, alpha=0.2)

plt.ylim(2.5, 0.9)

ax.set_ylabel("Airmass", fontsize="large")
ax.set_xlabel(f"Time ({tz})", fontsize="large")
ax_utc.set_xlabel("Time (UTC)", fontsize='large')
_ = plt.ylabel("Airmass", fontsize="large")

Visits from the ConsDB, as well as `Targets` and `Observations` from the EFD records generated from the FeatureBasedScheduler. <br>
The plot above stretches from sunset to sunrise, with civil, -12 degree and -18 degree sunset and sunrise indicated by the intensity of the gray shading. A blue (red) line indicates the time of moonrise (moonset), if applicable. 

Pink shaded regions (if present) indicate larger than expected overheads between visits (counted in the specified science programs only), and are numbered D-X. <br> 

A orange shaded region (if present) on the left of the plot indicates the time from sunset until the time the first `Target` is sent from the FBS.

`Targets` which have been linked with a corresponding `Observation` event (indicating the observing script completed successfully) are indicated by dots. `Targets` which were not able to be linked to an `Observation` are shown by pluses. 

`Visits` for `science` images are indicated by open circles, while `acq` `visits` are indicated by stars; `FOCUS` `visits` are shown with diamonds while `CWFS` `visits` are shown by hexagons.

Both `Targets`, `Observations` and `Visits` are color-coded by their either their science program or (when part of the main science program) their observation_reasons.

In [ ]:
display(Markdown("### Night report"))
report, html = endpoints['night_report'].query_night_report(day_obs, telescope=telescope)
display(HTML(html))

In [ ]:
# Pull up any time loss messages 
display(Markdown("### Time loss narrative log messages"))
ll = endpoints['narrative_log'].query_log(sunset, sunrise)
if salindex != 2:
    losses = ll.query("time_lost > 0 and not component.str.contains('AuxTel')")
else:
    losses = ll.query("time_lost > 0 and not component.str.contains('Simonyi')")
for i, loss in losses.iterrows():
    display(HTML(losses[i:i][['component', 'date_begin', 'date_end', 'date_added', "user_id", 'time_lost']].to_html()))
    print(loss['message_text'])

In [ ]:
if salindex != 2:
    from rubin_nights import scriptqueue
    messages, cols = scriptqueue.get_consolidated_messages(sunset, sunrise, endpoints, all_tracebacks=True)
    dumb_message = "Exception reading device RubinWiFiTest"
    messages = messages.query("not name.str.contains(@dumb_message)")
    dumb_message = "Failed to set guider ROI"
    messages = messages.query("not name.str.contains(@dumb_message)")

In [ ]:
display(Markdown("### messages during gaps"))
if 'category_index' in messages.columns:
    category_key = 'category_index' 
    categories = [1, 3, 4, 6, 7, 9, 10, 12, 20, 21]
else:
    category_key = 'salIndex'
    categories = [1, 3, 4, 5, 10, 12, 20, 22]
    
if salindex != 2:
    bcols = ["visit_id", "obs_start", "s_ra", "s_dec", "sky_rotation", "band", "altitude", "azimuth", "clouds", "fwhm_eff",
             "target_name", "observation_reason", "science_program", "slew_distance", "slew_model", "mt_slew_time", "visit_gap", "model_gap"]
    for i, b in enumerate(breaks):
        t_start = Time(b[0], format='mjd', scale='tai')
        t_start_dt = pd.to_datetime(t_start.utc.iso, utc=True)
        t_end = Time(b[1], format='mjd', scale='tai')
        t_end_dt = pd.to_datetime(t_end.utc.iso, utc=True)
        break_string = f"Break D-{i} from {t_start.isot} to {t_end.isot}"
        
        v1 = visitsS.query("obs_start_mjd <= @t_start.mjd").iloc[-1:]
        v2 = visitsS.query("obs_start_mjd >= @t_end.mjd").iloc[0:1]
        
        excess_gap = v2.model_gap.values[0] / 60
        
        break_string += f" ({(t_end - t_start).sec/60 :.2f} minutes gap; {v2.slew_model.values[0]/60 :.2f} minutes slew model; {excess_gap :.2f} excess gap)"
    
        if v1.band.values != v2.band.values:
            break_string += " FILTER CHANGE"
        if v1.science_program.values != v2.science_program.values: 
            break_string += " SCHEDULER CHANGE"
    
        t_start_nr = pd.to_datetime((t_start - TimeDelta(3*60, format='sec')).utc.iso, utc=True)
        t_end_nr = pd.to_datetime((t_end + TimeDelta(1*60, format='sec')).utc.iso, utc=True)

        nr_tr = messages.query("time >= @t_start_nr and time <= @t_end_nr")
        query = f"{category_key} in {categories}"
        nr_tr = nr_tr.query(query)
        print(break_string)  
        if (excess_gap > 1) | (len(nr_tr) > 0):
            display(pd.concat([v1, v2])[bcols])
            if len(nr_tr) > 0:
                display(HTML(nr_tr.to_html()))
            display(Markdown('---'))
            display(Markdown('---'))